# lgbm
time_diff 컬럼 넣어줌

In [0]:
import numpy as np 
import pandas as pd
from datetime import time, datetime

import os
from pathlib import Path


In [0]:
# 재생산성을 위해 시드 고정
np.random.seed(7)

http://machinelearningkorea.com/2019/09/29/lightgbm-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0/

# 1. 데이터 불러오기

In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [94]:
folder = "Colab Notebooks/dacon"
project_dir = "temperature_forecasting"
data_folder = "_data/data_dacon"

base_path = Path("/content/drive/My Drive/")
project_path = base_path / folder / project_dir
data_path = base_path / data_folder / project_dir
os.chdir(project_path)

print(f"현재 디렉토리 위치: {os.getcwd()}")

현재 디렉토리 위치: /content/drive/My Drive/Colab Notebooks/dacon/temperature_forecasting


In [95]:
df = pd.read_csv(os.path.join(data_path,'train.csv'), index_col = 'id')
df.shape

(4752, 59)

In [96]:
df_test = pd.read_csv(os.path.join(data_path,'test.csv'), index_col = 'id')
df_test.shape

(11520, 40)

#### 시간 정보 넣어주기

In [0]:
min_in_day = 24*6
hour_in_day = 24

feat_temp = ['X00', 'X07', 'X28', 'X31', 'X32']
diff = np.mean(df[feat_temp][:n], axis=1)- np.mean(df[['Y15', 'Y16', 'Y09']][:n], axis=1)
diff_14= {i:diff[i: :144].mean() for i in range(144)}

def time_data(frame):
    minute = (frame.index%144).astype(int)
    hour= pd.Series((frame.index%144/6).astype(int))

    minute_sin = np.sin(np.pi*minute/min_in_day) 
    minute_cos = np.cos(np.pi*minute/min_in_day)

    hour_sin  = np.sin(np.pi*hour/hour_in_day)
    hour_cos  = np.cos(np.pi*hour/hour_in_day)
    return minute_sin, minute_cos, hour_sin, hour_cos, (frame.index%144).map(diff_144)

In [98]:
# train data
print(df.shape)
df['time_m_sin'], df['time_m_cos'], df['time_h_sin'], df['time_h_cos'], df['time_diff'] = time_data(df)
print(df.shape)

(4752, 59)
(4752, 64)


In [99]:
# test data
print(df_test.shape)
df_test['time_m_sin'], df_test['time_m_cos'], df_test['time_h_sin'], df_test['time_h_cos'], df_test['time_diff'] = time_data(df_test)
print(df_test.shape)

(11520, 40)
(11520, 45)


In [0]:
n = 4320

|-|30일(train)|3일(train)|80일(test)|
|---|---|---|---|
|Y18|NULL|432개|NULL|
|Y00-Y17|4320개|NULL|NULL|
|X00-X17|4320개|432개|11520개|

In [0]:
df_30 = df[:n]
df_3 = df[n:n+n//10]

# x

* feat_temp: 기온 
* feat_air_press: 현지기압
* feat_wind_speed: 풍속
* feat_rainfall: 일일 누적강수량 
* feat_sea_press: 해면기압
* feat_insol: 일일 누적일사량
* feat_humid: 습도
* feat_dir_wind: 풍향



In [0]:
feat_time = ['time_m_sin', 'time_m_cos', 'time_h_sin', 'time_h_cos', 'time_diff']
feat_temp = ['X00', 'X07', 'X28', 'X31', 'X32']
feat_air_press = ['X01', 'X06', 'X22', 'X27', 'X29']
feat_wind_speed = ['X02', 'X03', 'X18', 'X24', 'X26']
feat_rainfall = ['X04', 'X10', 'X21', 'X36', 'X39']
feat_sea_press = ['X05', 'X08', 'X09', 'X23', 'X33']
feat_insol = ['X11', 'X14', 'X16', 'X19', 'X34']
feat_humid = ['X12', 'X20', 'X30', 'X37', 'X38']
feat_dir_wind = ['X13', 'X15', 'X17', 'X25', 'X35']

feat_x = ['X00', 'X01', 'X02', 'X03', 'X04', 'X05', 'X06', 'X07', 'X08', 'X09', 
          'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 
          'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 
          'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39']

feat_x_t = feat_x + feat_time 

### 일일누적일사량 missing data 삭제 
데이터가 의미 없는 열은 삭제 

In [0]:
remove_col =  ['X14', 'X16', 'X19']

In [0]:
for r in remove_col:
    feat_insol.remove(r)
    feat_x.remove(r)

In [0]:
df = df.drop(remove_col, axis = 1)

In [0]:
df_test = df_test.drop(remove_col, axis = 1)

## y

In [0]:
feat_y = ['Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 
          'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17']

# model

lgbm model - transfer learning (x로 yi평균 맞추게 하고 x로 y18 fine tuning)  
  
boosting_type 3가지 해봄. rf 는 계속 learning 할 수 없어서 못해봄 

In [0]:
import lightgbm as lgb
import collections
from sklearn.metrics import mean_squared_error

In [0]:
## loss function
def mse1(y_pred, dataset):
    y_true = dataset.get_label()

    diff = abs(y_true - y_pred)
    less_then_one = np.array([0 if x < 1 else 1 for x in diff])

    y_pred = less_then_one * y_pred
    y_true = less_then_one *y_true
    
    score= mean_squared_error(y_true, y_pred)
    
    return 'score', score, False

In [0]:
## data set 
## x_col, y_col 정하기 
x_col = feat_temp + feat_air_press+ feat_rainfall+ feat_insol+feat_dir_wind+feat_time
y_col = ['Y15', 'Y16', 'Y09'] 

data = df_30[x_col]
label = df_30[y_col].mean(axis=1)
m1_train_data = lgb.Dataset(data[:n*9//10], label=label[:n*9//10], free_raw_data=False)
m1_validation_data = lgb.Dataset(data[n*9//10:], label=label[n*9//10:], free_raw_data=False)

In [0]:
## data set (finetune)
y_col = ['Y18'] 
m = 432
data = df_3[x_col]
label = df_3[y_col]
m2_train_data = lgb.Dataset(data[:m*9//10], label=label[:m*9//10], free_raw_data=False)
m2_validation_data = lgb.Dataset(data[m*9//10:], label=label[m*9//10:], free_raw_data=False)

## model - lgbm 1 성능 8.36

In [0]:
## model parameter
params ={'boosting_type': 'gbdt',
 'learning_rate': 0.01,
 'num_leaves': 51,
 'random_state': 7,
 'objective': 'regression'}

In [129]:
model1 = lgb.train(params, 
            m1_train_data,
            feval=mse1,
            valid_sets=m1_validation_data,
            num_boost_round = 10000,
            verbose_eval=20,
            early_stopping_rounds=5,
            keep_training_booster=True) 

Training until validation scores don't improve for 5 rounds.
[20]	valid_0's l2: 25.8719	valid_0's score: 25.8431
[40]	valid_0's l2: 17.8237	valid_0's score: 17.7708
[60]	valid_0's l2: 12.4123	valid_0's score: 12.357
[80]	valid_0's l2: 8.699	valid_0's score: 8.62274
[100]	valid_0's l2: 6.17668	valid_0's score: 6.09617
[120]	valid_0's l2: 4.5081	valid_0's score: 4.41264
[140]	valid_0's l2: 3.40999	valid_0's score: 3.2806
[160]	valid_0's l2: 2.71128	valid_0's score: 2.55489
[180]	valid_0's l2: 2.22053	valid_0's score: 2.05222
[200]	valid_0's l2: 1.89949	valid_0's score: 1.71632
[220]	valid_0's l2: 1.66581	valid_0's score: 1.49193
[240]	valid_0's l2: 1.52364	valid_0's score: 1.34249
[260]	valid_0's l2: 1.43274	valid_0's score: 1.23058
[280]	valid_0's l2: 1.37122	valid_0's score: 1.17127
[300]	valid_0's l2: 1.34042	valid_0's score: 1.13871
[320]	valid_0's l2: 1.30706	valid_0's score: 1.09425
Early stopping, best iteration is:
[323]	valid_0's l2: 1.30419	valid_0's score: 1.09263


In [120]:
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            num_boost_round = 1000,
            verbose_eval = 5,
            early_stopping_rounds = 100,
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


Training until validation scores don't improve for 100 rounds.
[330]	valid_0's l2: 2.87796	valid_0's score: 2.82537
[335]	valid_0's l2: 2.89948	valid_0's score: 2.84549
[340]	valid_0's l2: 2.91942	valid_0's score: 2.7992
[345]	valid_0's l2: 2.95566	valid_0's score: 2.84116
[350]	valid_0's l2: 2.92994	valid_0's score: 2.84363
[355]	valid_0's l2: 2.89149	valid_0's score: 2.80926
[360]	valid_0's l2: 2.8606	valid_0's score: 2.78157
[365]	valid_0's l2: 2.83221	valid_0's score: 2.75541
[370]	valid_0's l2: 2.78917	valid_0's score: 2.69225
[375]	valid_0's l2: 2.74532	valid_0's score: 2.65199
[380]	valid_0's l2: 2.70142	valid_0's score: 2.61054
[385]	valid_0's l2: 2.66414	valid_0's score: 2.57614
[390]	valid_0's l2: 2.62046	valid_0's score: 2.53464
[395]	valid_0's l2: 2.5902	valid_0's score: 2.50617
[400]	valid_0's l2: 2.55401	valid_0's score: 2.44999
[405]	valid_0's l2: 2.5194	valid_0's score: 2.41929
[410]	valid_0's l2: 2.48508	valid_0's score: 2.36678
[415]	valid_0's l2: 2.44778	valid_0's sc

In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/lgbm_4_'+now+'.csv', index = False)

## model - lgbm dart 성능 8.99
하이퍼파라메터 조정  
boosting_type: dart  
'num_leaves': 200  

In [0]:
## model parameter
params ={'boosting_type': 'dart',
 'learning_rate': 0.01,
 'num_leaves': 200,
 'random_state': 7,
 'num_boost_round': 8800,
 'objective': 'regression'}

In [131]:
model1 = lgb.train(params, 
            m1_train_data,
            feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=100,
         #   early_stopping_rounds=5,
            keep_training_booster=True) 

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	valid_0's l2: 99.2708	valid_0's score: 99.2708
[200]	valid_0's l2: 78.7613	valid_0's score: 78.7613
[300]	valid_0's l2: 57.6933	valid_0's score: 57.6933
[400]	valid_0's l2: 59.2807	valid_0's score: 59.2807
[500]	valid_0's l2: 33.9725	valid_0's score: 33.9725
[600]	valid_0's l2: 31.433	valid_0's score: 31.433
[700]	valid_0's l2: 25.704	valid_0's score: 25.6998
[800]	valid_0's l2: 18.8165	valid_0's score: 18.8136
[900]	valid_0's l2: 12.9777	valid_0's score: 12.9672
[1000]	valid_0's l2: 9.90348	valid_0's score: 9.8847
[1100]	valid_0's l2: 10.2962	valid_0's score: 10.2833
[1200]	valid_0's l2: 14.4596	valid_0's score: 14.4478
[1300]	valid_0's l2: 10.4839	valid_0's score: 10.4695
[1400]	valid_0's l2: 6.80012	valid_0's score: 6.73828
[1500]	valid_0's l2: 5.88705	valid_0's score: 5.80508
[1600]	valid_0's l2: 5.1294	valid_0's score: 5.03466
[1700]	valid_0's l2: 4.26382	valid_0's score: 4.15658
[1800]	valid_0's l2: 4.13144	valid_0's score: 4.02125
[1900]	valid_0's l2: 4.37726	valid_0's sco

In [132]:
## model parameter
params ={'boosting_type': 'dart',
 'learning_rate': 0.01,
 'num_leaves': 200,
 'random_state': 7,
 'objective': 'regression'}
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            num_boost_round = 20000,
            verbose_eval = 100,
          #  early_stopping_rounds = 10,
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


[8900]	valid_0's l2: 2.4297	valid_0's score: 2.29776
[9000]	valid_0's l2: 1.99776	valid_0's score: 1.88711
[9100]	valid_0's l2: 1.71208	valid_0's score: 1.60933
[9200]	valid_0's l2: 1.6292	valid_0's score: 1.51837
[9300]	valid_0's l2: 1.70921	valid_0's score: 1.589
[9400]	valid_0's l2: 1.71156	valid_0's score: 1.61434
[9500]	valid_0's l2: 1.70219	valid_0's score: 1.60434
[9600]	valid_0's l2: 1.66629	valid_0's score: 1.57307
[9700]	valid_0's l2: 1.63547	valid_0's score: 1.54455
[9800]	valid_0's l2: 1.63612	valid_0's score: 1.52116
[9900]	valid_0's l2: 1.65865	valid_0's score: 1.56285
[10000]	valid_0's l2: 1.66342	valid_0's score: 1.58809
[10100]	valid_0's l2: 1.6837	valid_0's score: 1.60708
[10200]	valid_0's l2: 1.7058	valid_0's score: 1.62759
[10300]	valid_0's l2: 1.69635	valid_0's score: 1.61847
[10400]	valid_0's l2: 1.67447	valid_0's score: 1.59764
[10500]	valid_0's l2: 1.68408	valid_0's score: 1.60544
[10600]	valid_0's l2: 1.6955	valid_0's score: 1.61524
[10700]	valid_0's l2: 1.6996

In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/lgbm_4_'+now+'.csv', index = False)

## model - lgbm goss 성능 7.67 (lgbm_4_0316_1344, a4)

하이퍼파라메터 조정  
boosting_type: goss  
'num_leaves': 50  

In [0]:
## model parameter
params ={'boosting_type': 'goss',
 'learning_rate': 0.005,
 'num_leaves': 50,
 'random_state': 7,
 'objective': 'regression'}

In [139]:
model1 = lgb.train(params, 
            m1_train_data,
            feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=50,
            num_boost_round = 5000, 
            early_stopping_rounds=30,
            keep_training_booster=True) 

Training until validation scores don't improve for 30 rounds.
[50]	valid_0's l2: 23.5301	valid_0's score: 23.4885
[100]	valid_0's l2: 14.845	valid_0's score: 14.7881
[150]	valid_0's l2: 9.53702	valid_0's score: 9.46379
[200]	valid_0's l2: 6.20007	valid_0's score: 6.12078
[250]	valid_0's l2: 2.48092	valid_0's score: 2.29777
[300]	valid_0's l2: 1.8623	valid_0's score: 1.66376
[350]	valid_0's l2: 1.59326	valid_0's score: 1.38149
[400]	valid_0's l2: 1.49202	valid_0's score: 1.2815
[450]	valid_0's l2: 1.43991	valid_0's score: 1.25759
[500]	valid_0's l2: 1.39906	valid_0's score: 1.22369
[550]	valid_0's l2: 1.37792	valid_0's score: 1.19794
[600]	valid_0's l2: 1.36027	valid_0's score: 1.17349
[650]	valid_0's l2: 1.33767	valid_0's score: 1.15859
[700]	valid_0's l2: 1.34212	valid_0's score: 1.17124
Early stopping, best iteration is:
[681]	valid_0's l2: 1.26299	valid_0's score: 1.0816


In [145]:
## model parameter
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            verbose_eval = 20,
            num_boost_round = 5000,             
            early_stopping_rounds = 100,
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


Training until validation scores don't improve for 100 rounds.
[720]	valid_0's l2: 2.90185	valid_0's score: 2.8481
[740]	valid_0's l2: 2.93142	valid_0's score: 2.81531
[760]	valid_0's l2: 2.90782	valid_0's score: 2.82422
[780]	valid_0's l2: 2.85035	valid_0's score: 2.77339
[800]	valid_0's l2: 2.77278	valid_0's score: 2.67828
[820]	valid_0's l2: 2.68521	valid_0's score: 2.59629
[840]	valid_0's l2: 2.59753	valid_0's score: 2.51365
[860]	valid_0's l2: 2.5307	valid_0's score: 2.4293
[880]	valid_0's l2: 2.46808	valid_0's score: 2.35061
[900]	valid_0's l2: 2.41235	valid_0's score: 2.2785
[920]	valid_0's l2: 1.77023	valid_0's score: 1.66209
[940]	valid_0's l2: 2.23695	valid_0's score: 2.11467
[960]	valid_0's l2: 2.16602	valid_0's score: 2.03303
[980]	valid_0's l2: 2.03295	valid_0's score: 1.8908
[1000]	valid_0's l2: 2.2579	valid_0's score: 2.15914
Early stopping, best iteration is:
[913]	valid_0's l2: 1.58793	valid_0's score: 1.4581


In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/lgbm_4_'+now+'.csv', index = False)

## model - lgbm goss - 2 성능 5.76 (0316_1356, a6)
하이퍼파라메터 조정 
boosting_type: goss  
'num_leaves': 100  

In [0]:
## model parameter
params ={'boosting_type': 'goss',
 'learning_rate': 0.0005,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [149]:
model1 = lgb.train(params, 
            m1_train_data,
            feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=100,
            num_boost_round = 5000, 
            early_stopping_rounds=30,
            keep_training_booster=True) 

Training until validation scores don't improve for 30 rounds.
[100]	valid_0's l2: 34.5206	valid_0's score: 34.4872
[200]	valid_0's l2: 31.3913	valid_0's score: 31.3538
[300]	valid_0's l2: 28.5697	valid_0's score: 28.5303
[400]	valid_0's l2: 26.0096	valid_0's score: 25.9717
[500]	valid_0's l2: 23.6965	valid_0's score: 23.6573
[600]	valid_0's l2: 21.5633	valid_0's score: 21.5218
[700]	valid_0's l2: 19.6403	valid_0's score: 19.5939
[800]	valid_0's l2: 17.9117	valid_0's score: 17.8638
[900]	valid_0's l2: 16.3632	valid_0's score: 16.3118
[1000]	valid_0's l2: 14.9481	valid_0's score: 14.8904
[1100]	valid_0's l2: 13.6784	valid_0's score: 13.618
[1200]	valid_0's l2: 12.5341	valid_0's score: 12.4786
[1300]	valid_0's l2: 11.4994	valid_0's score: 11.4412
[1400]	valid_0's l2: 10.5507	valid_0's score: 10.5015
[1500]	valid_0's l2: 9.68535	valid_0's score: 9.63713
[1600]	valid_0's l2: 8.91092	valid_0's score: 8.85704
[1700]	valid_0's l2: 8.21071	valid_0's score: 8.15245
[1800]	valid_0's l2: 7.57923	v

In [0]:
## model parameter
params ={'boosting_type': 'goss',
 'learning_rate': 0.0001,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [152]:
## model parameter
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            verbose_eval = 20,
            early_stopping_rounds = 50,
            num_boost_round = 10000, 
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


Training until validation scores don't improve for 50 rounds.
[4540]	valid_0's l2: 2.85642	valid_0's score: 2.8053
[4560]	valid_0's l2: 2.85916	valid_0's score: 2.80791
[4580]	valid_0's l2: 2.86191	valid_0's score: 2.81051
Early stopping, best iteration is:
[4537]	valid_0's l2: 2.856	valid_0's score: 2.80491


In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/lgbm_4_'+now+'.csv', index = False)

## model - lgbm goss - 3 성능 5.78 (0316_1358, a8)
하이퍼파라메터 조정  
metric: l2 (rmse랑 같음) 
boosting_type: goss  
'num_leaves': 100


In [0]:
## model parameter
params ={'metric': 'l2',
 'boosting_type': 'goss',
 'learning_rate': 0.0005,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [157]:
model1 = lgb.train(params, 
            m1_train_data,
           # feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=100,
            num_boost_round = 10000, 
            early_stopping_rounds=30,
            keep_training_booster=True) 

Training until validation scores don't improve for 30 rounds.
[100]	valid_0's l2: 34.5206
[200]	valid_0's l2: 31.3913
[300]	valid_0's l2: 28.5697
[400]	valid_0's l2: 26.0096
[500]	valid_0's l2: 23.6965
[600]	valid_0's l2: 21.5633
[700]	valid_0's l2: 19.6403
[800]	valid_0's l2: 17.9117
[900]	valid_0's l2: 16.3632
[1000]	valid_0's l2: 14.9481
[1100]	valid_0's l2: 13.6784
[1200]	valid_0's l2: 12.5341
[1300]	valid_0's l2: 11.4994
[1400]	valid_0's l2: 10.5507
[1500]	valid_0's l2: 9.68535
[1600]	valid_0's l2: 8.91092
[1700]	valid_0's l2: 8.21071
[1800]	valid_0's l2: 7.57923
[1900]	valid_0's l2: 7.00904
[2000]	valid_0's l2: 6.49456
[2100]	valid_0's l2: 4.88492
[2200]	valid_0's l2: 3.95411
[2300]	valid_0's l2: 3.37098
[2400]	valid_0's l2: 2.98259
[2500]	valid_0's l2: 2.6952
[2600]	valid_0's l2: 2.4895
[2700]	valid_0's l2: 2.31848
[2800]	valid_0's l2: 2.17795
[2900]	valid_0's l2: 2.05788
[3000]	valid_0's l2: 1.9605
[3100]	valid_0's l2: 1.87148
[3200]	valid_0's l2: 1.79194
[3300]	valid_0's l2: 1

In [0]:
## model parameter
params ={'boosting_type': 'goss',
 'learning_rate': 0.0001,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [159]:
## model parameter
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            verbose_eval = 20,
            early_stopping_rounds = 50,
            num_boost_round = 20000, 
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


Training until validation scores don't improve for 50 rounds.
[5360]	valid_0's l2: 2.85751	valid_0's score: 2.80634
[5380]	valid_0's l2: 2.86026	valid_0's score: 2.80895
Early stopping, best iteration is:
[5349]	valid_0's l2: 2.856	valid_0's score: 2.80491


In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/lgbm_4_'+now+'.csv', index = False)

## model - lgbm goss - 4 (0315-1029) 성능 X
(3과 거의 비슷해서 안해봐도 될듯) 
하이퍼파라메터 조정  

metric: mape
boosting_type: goss  
'num_leaves': 100


In [0]:
## model parameter
params ={'metric': 'mape',
 'boosting_type': 'goss',
 'learning_rate': 0.0005,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [163]:
model1 = lgb.train(params, 
            m1_train_data,
           # feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=100,
            num_boost_round = 10000, 
            early_stopping_rounds=30,
            keep_training_booster=True) 

Training until validation scores don't improve for 30 rounds.
[100]	valid_0's mape: 0.212876
[200]	valid_0's mape: 0.202849
[300]	valid_0's mape: 0.193427
[400]	valid_0's mape: 0.184569
[500]	valid_0's mape: 0.176264
[600]	valid_0's mape: 0.168296
[700]	valid_0's mape: 0.160748
[800]	valid_0's mape: 0.153617
[900]	valid_0's mape: 0.146914
[1000]	valid_0's mape: 0.140554
[1100]	valid_0's mape: 0.134531
[1200]	valid_0's mape: 0.128826
[1300]	valid_0's mape: 0.12343
[1400]	valid_0's mape: 0.118264
[1500]	valid_0's mape: 0.113379
[1600]	valid_0's mape: 0.108846
[1700]	valid_0's mape: 0.104597
[1800]	valid_0's mape: 0.100604
[1900]	valid_0's mape: 0.0967606
[2000]	valid_0's mape: 0.0931056
[2100]	valid_0's mape: 0.0751676
[2200]	valid_0's mape: 0.0633474
[2300]	valid_0's mape: 0.0556192
[2400]	valid_0's mape: 0.0506941
[2500]	valid_0's mape: 0.0471789
[2600]	valid_0's mape: 0.0449986
[2700]	valid_0's mape: 0.0432382
[2800]	valid_0's mape: 0.0417289
[2900]	valid_0's mape: 0.0404635
[3000]	va

In [0]:
## model parameter
params['learning_rate'] =  params['learning_rate']/5

In [165]:
## model parameter
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            verbose_eval = 10,
            early_stopping_rounds = 50,
            num_boost_round = 20000, 
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


Training until validation scores don't improve for 50 rounds.
[4460]	valid_0's mape: 0.066124	valid_0's score: 2.80517
[4470]	valid_0's mape: 0.0661385	valid_0's score: 2.80647
[4480]	valid_0's mape: 0.0661531	valid_0's score: 2.80778
[4490]	valid_0's mape: 0.0661676	valid_0's score: 2.80908
[4500]	valid_0's mape: 0.0661821	valid_0's score: 2.81038
Early stopping, best iteration is:
[4458]	valid_0's mape: 0.0661211	valid_0's score: 2.80491


In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/baseline_result_2_'+now+'.csv', index = False)

# tmp
